# DataQuest Guided Project
## Preparing data for SQLite

In [1]:
import pandas as pd
import sqlite3

In [14]:
df = pd.read_csv('academy_awards.csv', encoding='ISO-8859-1')
df.head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010 (83rd),Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010 (83rd),Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010 (83rd),Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN
3,2010 (83rd),Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN
4,2010 (83rd),Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df['Unnamed: 5'].value_counts()

*                                                                                                               7
 error-prone measurements on sets. [Digital Imaging Technology]"                                                1
 D.B. "Don" Keele and Mark E. Engebretson has resulted in the over 20-year dominance of constant-directivity    1
 discoverer of stars                                                                                            1
 resilience                                                                                                     1
Name: Unnamed: 5, dtype: int64

### Filtering the data

In [15]:
df["Year"] = df["Year"].str[0:4]
df["Year"] = df["Year"].astype("int64")
later_than_2000 = df[df["Year"]>2000]

award_categories = ["Actor -- Leading Role", "Actor -- Supporting Role", "Actress -- Leading Role", "Actress -- Supporting Role"]
nominations = later_than_2000[later_than_2000["Category"].isin(award_categories)]


### Cleaning up the Won? column

In [16]:
replace_dict = { "NO": 0, "YES": 1}
nominations["Won?"] = nominations["Won?"].map(replace_dict)

nominations["Won"] = nominations["Won?"]

redundant = ["Won?", "Unnamed: 5", "Unnamed: 6", "Unnamed: 7", "Unnamed: 8", "Unnamed: 9", "Unnamed: 10"]
final_nominations = nominations.drop(redundant, axis=1)

C:\Users\rosie.tulloch\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\rosie.tulloch\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Cleaning up the Additional Info column

In [19]:
a1 = final_nominations["Additional Info"].str.rstrip("'}")
a2 = a1.str.split(" {'")
final_nominations["Character"] = a2.str[1]
final_nominations["Movie"] = a2.str[0]
final_nominations = final_nominations.drop("Additional Info", axis=1)

### Exporting to SQLite

In [20]:
import sqlite3
conn = sqlite3.connect("nominations.db")

In [21]:
final_nominations.to_sql("nominations", conn, index=False)

### Verifying in SQL

In [23]:
conn.execute("PRAGMA table_info(nominations)").fetchall()

[(0, 'Year', 'INTEGER', 0, None, 0),
 (1, 'Category', 'TEXT', 0, None, 0),
 (2, 'Nominee', 'TEXT', 0, None, 0),
 (3, 'Won', 'INTEGER', 0, None, 0),
 (4, 'Character', 'TEXT', 0, None, 0),
 (5, 'Movie', 'TEXT', 0, None, 0)]

In [25]:
response = conn.execute("SELECT * FROM nominations LIMIT 10").fetchall()

In [26]:
conn.close

<function Connection.close>

### Further exploration

In [27]:
final_nominations.head()

,Year,Category,Nominee,Won,Character,Movie
0,2010,Actor -- Leading Role,Javier Bardem,0,Uxbal,Biutiful
1,2010,Actor -- Leading Role,Jeff Bridges,0,Rooster Cogburn,True Grit
2,2010,Actor -- Leading Role,Jesse Eisenberg,0,Mark Zuckerberg,The Social Network
3,2010,Actor -- Leading Role,Colin Firth,1,King George VI,The King's Speech
4,2010,Actor -- Leading Role,James Franco,0,Aron Ralston,127 Hours
